In [218]:
# # Installing required packages
# !pip install pyspark
# !pip install findspark
# !pip install pyarrow==1.0.0
# !pip install pandas
# !pip install numpy==1.19.5

In [219]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [220]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Get the existing SparkContext
sc = SparkContext.getOrCreate()

# Create a SparkSession using the existing SparkContext
spark = SparkSession.builder.appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [221]:
spark

In [222]:
df = pd.read_csv('/content/music_streaming.csv')
df.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15517 entries, 0 to 15516
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         15517 non-null  object 
 1   Track Name          15517 non-null  object 
 2   Popularity          15123 non-null  float64
 3   danceability        15517 non-null  float64
 4   energy              15517 non-null  float64
 5   key                 13774 non-null  float64
 6   loudness            15517 non-null  float64
 7   mode                15517 non-null  int64  
 8   speechiness         15517 non-null  float64
 9   acousticness        15517 non-null  float64
 10  instrumentalness    11930 non-null  float64
 11  liveness            15517 non-null  float64
 12  valence             15517 non-null  float64
 13  tempo               15517 non-null  float64
 14  duration_in min/ms  15517 non-null  float64
 15  time_signature      15517 non-null  int64  
 16  Genr

# Data Processing

### Missing Values

In [224]:
# 2. missing values in each column
df.isnull().sum()

Artist Name              0
Track Name               0
Popularity             394
danceability             0
energy                   0
key                   1743
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      3587
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Genre                    0
dtype: int64

In [225]:
#drop rows where Popularity is missing as there are 352 only missing
df = df.dropna(subset=['Popularity'])
df['Popularity'].isnull().sum()

0

In [226]:
#drop key column as it is not needed in our analysis
df = df.drop(columns=['key'])
df.head()

,Artist Name,Track Name,Popularity,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


Here we are calculating the mean of the 'instrumentalness' column for each group of rows grouped by 'Artist Name'.
Then, we fill the missing (NaN) values in the 'instrumentalness' column with the mean of each artist's instrumentalness.

In [227]:
df['instrumentalness'] = df.groupby('Artist Name')['instrumentalness'].transform(lambda x: x.fillna(x.mean()))
df['instrumentalness'].isnull().sum()

2022

I fill missing values in the 'instrumentalness' column of my DataFrame with the mean of all non-missing values in that column and then count how many missing values remain.

In [228]:
# impute instrumentalness with the mean of the instrumentalness of all songs
df['instrumentalness'] = df['instrumentalness'].fillna(df['instrumentalness'].mean())
df['instrumentalness'].isnull().sum()

0

In [229]:
df.isnull().sum()

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Genre                 0
dtype: int64

### Duplicates

In [230]:
# Before removing duplicates
rows_before = df.shape[0]

# Drop duplicate rows based on both 'Track Name' and 'Artist Name'
df_no_duplicates = df.drop_duplicates(subset=["Track Name", "Artist Name"])

# Drop duplicate rows based on 'Track Name' alone
df_no_duplicates = df_no_duplicates.drop_duplicates(subset=["Track Name"])

# After removing duplicates
rows_after = df_no_duplicates.shape[0]

print("Number of rows before removing duplicates:", rows_before)
print("Number of rows after removing duplicates:", rows_after)


Number of rows before removing duplicates: 15123
Number of rows after removing duplicates: 12712


### Transformation

Transform duration to minutes only

In [231]:
df[df['duration_in min/ms'] < 1000]['duration_in min/ms'].sort_values(ascending=False)

13613    29.886000
1108     29.450000
13007    23.291550
5556     21.592667
9881     18.709783
           ...    
5640      0.987117
11302     0.979333
8830      0.969150
1360      0.533917
4184      0.501650
Name: duration_in min/ms, Length: 2166, dtype: float64

In [232]:
# maximum song length in minutes is 29.8
# `duration_in min/ms` > 1000 then divide the value by 60000 to convert it to minutes
# we are sure that the all the values > 29.8 are in milliseconds

df['duration_in min/ms'] = df['duration_in min/ms'].apply(lambda x: x/60000 if x > 1000 else x)

In [233]:
sdf = spark.createDataFrame(df)
sdf.printSchema()

root
 |-- Artist Name: string (nullable = true)
 |-- Track Name: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_in min/ms: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- Genre: long (nullable = true)



In [234]:
sdf.createOrReplaceTempView("Spotify_streaming")

In [235]:
# Showing the whole table
spark.sql("SELECT * FROM Spotify_streaming").show()

+--------------------+--------------------+----------+------------+------+--------+----+-----------+------------+--------------------+--------+-------+-------+------------------+--------------+-----+
|         Artist Name|          Track Name|Popularity|danceability|energy|loudness|mode|speechiness|acousticness|    instrumentalness|liveness|valence|  tempo|duration_in min/ms|time_signature|Genre|
+--------------------+--------------------+----------+------------+------+--------+----+-----------+------------+--------------------+--------+-------+-------+------------------+--------------+-----+
|          Bruno Mars|That's What I Lik...|      60.0|       0.854| 0.564|  -4.964|   1|     0.0485|      0.0171| 0.16094874956104677|  0.0849|  0.899|134.071|3.9099333333333335|             4|    5|
|              Boston|        Hitch a Ride|      54.0|       0.382| 0.814|   -7.23|   1|     0.0406|      0.0011|             0.00401|   0.101|  0.569|116.454|           4.19555|             4|   10|


### a) Which genre has the highest average popularity?

In [236]:
# the average popularity for each genre
sql_query = """
SELECT Genre, AVG(Popularity) AS AvgPopularity
FROM Spotify_streaming
GROUP BY Genre
ORDER BY AvgPopularity DESC
LIMIT 1
"""

# Execute query and results
result = spark.sql(sql_query)
result.show()

+-----+-----------------+
|Genre|    AvgPopularity|
+-----+-----------------+
|    4|57.36974789915966|
+-----+-----------------+



### b) Display which artists have recorded the most number of songs with a duration of more than 5 minutes ?

In [237]:
# filter songs with a duration of more than 5 minutes, group by artist, and count the number of songs for each artist
sql_query = """
SELECT `Artist Name`, COUNT(*) AS NumSongs
FROM Spotify_streaming
WHERE `duration_in min/ms` > 5
GROUP BY `Artist Name`
ORDER BY NumSongs DESC
"""

result = spark.sql(sql_query)
result.show()

+--------------------+--------+
|         Artist Name|NumSongs|
+--------------------+--------+
|           Metallica|      21|
|                TOOL|      13|
|         Arcade Fire|      12|
|        Led Zeppelin|      12|
|           Aerosmith|      10|
|           Pearl Jam|       9|
|         Asha Bhosle|       8|
|        Dire Straits|       7|
|               Kyuss|       6|
|Kenny Wayne Shepherd|       6|
|       Wooden Shjips|       6|
|       Mohammed Rafi|       6|
|         Sonic Youth|       6|
|     Lata Mangeshkar|       6|
|             Pantera|       6|
|       Joe Bonamassa|       6|
|       Guns N' Roses|       6|
|   Avenged Sevenfold|       5|
|      Monster Magnet|       5|
|             Advaita|       5|
+--------------------+--------+
only showing top 20 rows



### c) How many songs are included in every Genre?

In [238]:
# This query group the songs by genre and count the number of songs for each genre
sql_query = """
SELECT Genre, COUNT(*) AS NumSongs
FROM Spotify_streaming
GROUP BY Genre
ORDER BY NumSongs DESC
"""

result = spark.sql(sql_query)
result.show()

+-----+--------+
|Genre|NumSongs|
+-----+--------+
|   10|    4185|
|    6|    2223|
|    9|    1768|
|    8|    1675|
|    1|    1241|
|    5|    1189|
|    2|    1169|
|    0|     517|
|    7|     441|
|    3|     358|
|    4|     357|
+-----+--------+



### d) Which artists dominated the charts?

In [239]:
# calculate the total popularity for each artist
sql_query = """
SELECT `Artist Name`, AVG(Popularity) AS AveragePopularity
FROM Spotify_streaming
GROUP BY `Artist Name`
ORDER BY AveragePopularity DESC
"""
result = spark.sql(sql_query)
result.show()

+--------------------+-----------------+
|         Artist Name|AveragePopularity|
+--------------------+-----------------+
|            Måneskin|            100.0|
|The Kid LAROI, Ju...|             97.0|
|       Doja Cat, SZA|             95.0|
|Justin Bieber, Da...|             95.0|
|     Los Legendarios|             95.0|
|The Weeknd, Arian...|             94.0|
|          Nio Garcia|             93.0|
|Riton, Nightcrawl...|             92.0|
|                 ATB|             91.0|
|          Tion Wayne|             90.0|
|             Cardi B|             90.0|
|    Dua Lipa, DaBaby|             90.0|
|          The Weeknd|            89.25|
|Galantis, David G...|             89.0|
|  Majestic, Boney M.|             89.0|
|                 SZA|             89.0|
|        Nathan Evans|             89.0|
|     ATB, Topic, A7S|             88.0|
|    Trinidad Cardona|             88.0|
|            Majestic|             88.0|
+--------------------+-----------------+
only showing top

### e) Recommend at least 5 fun/not-boring songs that can be played at a party, you can use features like energy, danceability etc.. to represent cheerfulness.


In [241]:
# songs with high danceability: Describes how suitable a track is for dancing
# high tempo: is the speed or pace of a given piece
# high valence: Tracks with high valence sound more positive
spark.sql("SELECT `Track Name`, `Artist Name`, danceability, tempo, energy, valence FROM Spotify_streaming ORDER BY danceability DESC, tempo DESC, energy DESC, valence DESC LIMIT 5").show()

+--------------------+---------------+------------+-------+------+-------+
|          Track Name|    Artist Name|danceability|  tempo|energy|valence|
+--------------------+---------------+------------+-------+------+-------+
|      Gucci Umbrella|Whookilledkenny|       0.989|120.004|  0.54|  0.604|
|Divine Gosa - Swi...|      Radioclit|       0.982|131.023| 0.641|  0.866|
|Divine Gosa - Swi...|      Radioclit|       0.982|131.023| 0.641|  0.866|
|           kawamurra|        sunflwr|        0.98|126.004| 0.529|  0.285|
|  Dancing in My Room|       347aidan|        0.98|119.993| 0.413|  0.764|
+--------------------+---------------+------------+-------+------+-------+



## ML MODELS

### Preparing Data for Machine Learning

In [242]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.sql import DataFrame

# Define numeric columns and featurization pipeline
numericCols = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
featurizationPipeline = Pipeline(stages=[VectorAssembler(inputCols=numericCols, outputCol="features")])

# Fit the featurization pipeline to the data
featurizationPipelineModel = featurizationPipeline.fit(sdf)
sdf = featurizationPipelineModel.transform(sdf)

# Split the data into training and testing sets
train_data, test_data = sdf.randomSplit([0.8, 0.2], seed=42)

train_data.createOrReplaceTempView("train_data_view")
test_data.createOrReplaceTempView("test_data_view")

### Logistic Regression Model

In [243]:
from pyspark.ml.classification import LogisticRegression

# Create a Logistic Regression classifier
lr = LogisticRegression(featuresCol='features', labelCol='Genre')

# Fit the model to the training data
lr_model = lr.fit(train_data)

# Make predictions on the test data
lr_predictions = lr_model.transform(test_data)

### Random Forest Model

In [244]:
from pyspark.ml.classification import RandomForestClassifier

# Create a Random Forest classifier
rf = RandomForestClassifier(featuresCol='features', labelCol='Genre')

# Fit the model to the training data
rf_model = rf.fit(train_data)

# Make predictions on the test data
rf_predictions = rf_model.transform(test_data)

### Decision Tree Model

In [245]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create a Decision Tree classifier
dt = DecisionTreeClassifier(featuresCol='features', labelCol='Genre')

# Fit the model to the training data
dt_model = dt.fit(train_data)

# Make predictions on the test data
dt_predictions = dt_model.transform(test_data)

# Models Results

In [246]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create an evaluator for accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='Genre', predictionCol='prediction', metricName='accuracy')

#  Logistic Regression
lr_accuracy = evaluator.evaluate(lr_predictions)

#  Decision Tree Classifier
dt_accuracy = evaluator.evaluate(dt_predictions)

#  Random Forest Classifier
rf_accuracy = evaluator.evaluate(rf_predictions)

# Print the accuracies
print("Logistic Regression Accuracy:", lr_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("Decision Tree Accuracy:", dt_accuracy)

Logistic Regression Accuracy: 0.41423728813559324
Random Forest Accuracy: 0.40644067796610167
Decision Tree Accuracy: 0.3888135593220339


### Insights


Logistic Regression achieved the highest accuracy (41.36%) in predicting song genres based on provided features, outperforming Random Forest (39.86%) and Decision Tree (39.19%) models.